In [1]:
from google.colab import drive
import os
import time
import pandas as pd
import numpy as np
import random

In [2]:
# Mounting Google Drive Location of training images
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/FourthBrain/Capstone Samsung OCT/')

Mounted at /content/drive


In [32]:
# Setting up the variables of where the processing will occur
sinGAN_dir = "./sinGAN-tf/"
data_dir = "./Data/OCT2017/"
synthetic_dir = data_dir + "sinGAN-Data/synthetic/"
sinGAN_model = sinGAN_dir + "main.py"
sinGAN_chkpt = sinGAN_dir + "training_checkpoints/"

# You just need to set the class you are training for, set it to all CAPITAL
train_class = "CNV"
sinGAN_train_dir = data_dir + "sinGAN-Data/" + train_class

#  Change this value to True only if you have new images that you want to train on, otherwise leave to FALSE
REGEN = False

In [4]:
def create_df (path, sub_dir):
  df = pd.DataFrame(columns=['filename', 'cnv', 'dme', 'drusen', 'normal'])
  condition = {'normal': 0, 'cnv': 0, 'dme':0, 'drusen': 0}
  files = os.listdir(path + sub_dir)
  if (sub_dir == 'NORMAL'):
    condition['normal'] = 1
  elif (sub_dir == 'CNV'):
    condition['cnv'] = 1
  elif (sub_dir == 'DME'):
    condition['dme'] = 1
  else:
    condition['drusen']= 1
  for f in files:
    df = df.append({'filename': path +  sub_dir  + "/" + f,  
                    'cnv': condition['cnv'],
                    'dme': condition['dme'],
                    'drusen': condition['drusen'],
                    'normal': condition['normal']}, ignore_index=True)
  return df

In [5]:
classes = ['NORMAL', 'CNV', 'DME', 'DRUSEN']
cols = [x.lower() for x in classes]
label = {0: 'CNV', 1: 'DME', 2: 'DRUSEN', 3: 'NORMAL'}

if (REGEN):
  for directory in classes:
    train_df = create_df(data_dir + "sinGAN-Data/", directory)
    train_df.set_index('filename', inplace=True)
    train_df.to_csv(data_dir + "sinGAN-Data/" + directory + "_train.csv")
else:
  train_df = pd.read_csv(sinGAN_train_dir + "_train.csv")

In [21]:
def train_sinGAN(filename):
  !python $sinGAN_model train --image $filename --num_scales 8

In [47]:
def pick_random(filename):
  !python $sinGAN_model inference --image $filename  --dir $sinGAN_chkpt  --mode random_sample  --inject_scale 0
  rand_num = random.sample(range(0,49),20)
  print (rand_num)
  for i in rand_num:
    os.replace("./results/random_sample/random_sample_" + str(i) + ".jpg", synthetic_dir + train_class + "/" + str(i) + "_"  + os.path.basename(filename))

In [48]:
for row in train_df["filename"]:
  #train_sinGAN(row)
  pick_random(row)
  print (row)
  break

./Data/OCT2017/sinGAN-Data/CNV/CNV-103044-193.jpeg
2021-08-23 01:27:29.020845: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-23 01:27:29.029388: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-23 01:27:29.030052: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-23 01:27:29.030888: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebu

In [9]:
os.getcwd()

'/content/drive/My Drive/FourthBrain/Capstone Samsung OCT'

In [17]:
os.chdir(sinGAN_dir)

dir_path = "./images/batch/"

for f in os.listdir(dir_path):
  file_path = dir_path + f
  print (file_path)
  !python main.py inference --image file_path  --dir ./training_checkpoints  --mode random_sample  --inject_scale 0

In [18]:
!python main.py inference --image NORMAL-220042-1.jpeg  --dir ./training_checkpoints  --mode random_sample  --inject_scale 0 --num_samples 1

NORMAL-220042-1.jpeg
Traceback (most recent call last):
  File "main.py", line 98, in <module>
    main()
  File "main.py", line 69, in main
    assert os.path.exists(args.image), 'Reference image not found !'
AssertionError: Reference image not found !


In [19]:
def infer_images (image):
  !python main.py inference --image $image  --dir ./training_checkpoints  --mode random_sample  --inject_scale 0
  time.sleep(5)
  return

In [ ]:
dir_path = "images/batch/"

for f in os.listdir(dir_path): 
  file_path = dir_path + f 
  print (file_path)
  infer_images (file_path)
  #assert os.path.exists(file_path), "Hello***"
  break

images/batch/CNV-6875371-1.jpeg


In [ ]:
os.getcwd()